In [1]:
import pandas as pd
import numpy as np

#Modify the csv file to remove crime records that are missing geolocation
crime_df = pd.read_csv("../LA_Data/LA_Crimes_2010_to_4_21_2019.csv")
#rental_df = pd.read_csv("../Rental_Data/Single_Family_Residence_Rental.csv", encoding = "ISO-8859-1")
#rental_df = rental_df[rental_df['City'] == 'Los Angeles']
#rental_df.set_index('RegionName', inplace = True)
crime_df

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,161816333,08/19/2016,08/18/2016,2140,18,Southeast,1844,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 1300,...,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,11200 STANFORD AV,NaN,"(33.9328, -118.2621)"
1,161816334,08/19/2016,08/18/2016,1600,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,87TH ST,AVALON BL,"(33.9581, -118.2651)"
2,161816335,08/19/2016,08/18/2016,2330,18,Southeast,1804,330,BURGLARY FROM VEHICLE,0344 1601,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,700 CAMELOT WY,NaN,"(33.9573, -118.263)"
3,161816337,08/19/2016,08/18/2016,2140,18,Southeast,1863,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1300,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,300 E 116TH ST,NaN,"(33.9292, -118.2695)"
4,161816338,08/19/2016,08/18/2016,2130,18,Southeast,1891,330,BURGLARY FROM VEHICLE,0344 1609 1307,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,16700 S FIGUEROA ST,NaN,"(33.8773, -118.2867)"
5,161816339,08/19/2016,08/19/2016,100,18,Southeast,1846,626,INTIMATE PARTNER - SIMPLE ASSAULT,2000 0400,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,626.0,NaN,NaN,NaN,1400 E 115TH ST,NaN,"(33.9304, -118.2504)"
6,161816342,08/19/2016,08/19/2016,1220,18,Southeast,1831,210,ROBBERY,0334 0337 0344,...,REVOLVER,AA,Adult Arrest,210.0,NaN,NaN,NaN,12700 S FIGUEROA ST,NaN,"(33.9176, -118.2828)"
7,161816344,08/19/2016,08/19/2016,1840,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 94TH ST,NaN,"(33.9511, -118.2717)"
8,161816345,08/19/2016,08/19/2016,715,18,Southeast,1803,210,ROBBERY,0344 1822 1300,...,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,210.0,NaN,NaN,NaN,300 E MANCHESTER AV,NaN,"(33.9601, -118.2695)"
9,161816346,08/19/2016,08/19/2016,1800,18,Southeast,1844,210,ROBBERY,0344 0216,...,SEMI-AUTOMATIC PISTOL,AA,Adult Arrest,210.0,NaN,NaN,NaN,600 E IMPERIAL HY,NaN,"(33.9345, -118.2623)"


In [2]:
crime_df['Longitude'] = crime_df['Location '].apply(lambda x: float(x.split(',')[1].split(')')[0]))
crime_df['Latitude'] = crime_df['Location '].apply(lambda x: float(x.split(',')[0].split('(')[1]))
crime_df['Geolocation'] = list(zip(crime_df['Latitude'], crime_df['Longitude']))
crime_df = crime_df.loc[crime_df['Latitude']!=0,:]

In [ ]:
crime_df.loc[crime_df['Latitude']== np.nan,:]

In [6]:
#This code assigned zipcodes to crime records based on geolocation
from uszipcode import SearchEngine
def getZip(x):
    try:
        return search.by_coordinates(x[0], x[1], radius=5, returns=1)[0].zipcode
    except:
        return np.nan
search = SearchEngine(simple_zipcode=True)
crime_df['Zipcode'] = crime_df['Geolocation'].apply(lambda x: getZip(x))
crime_df.to_csv('../LA_Data/LA_Crimes_Geolocations', index=False)

In [50]:
crime_df['Year-Month'] = crime_df['Date Occurred'].apply(lambda x: x.split('/')[2]+'-'+x.split('/')[0])
crime_df['Year'] = crime_df['Date Occurred'].apply(lambda x: int(x.split('/')[2]))
slice_df = crime_df.loc[crime_df['Year'] >= 2011, :]
slice_df
slice_df['Tuple'] = list(zip(slice_df['Zipcode'], slice_df['Year-Month']))
zipcodes = list(map(str,rental_df.index.unique()))
slice_zip = slice_df.loc[slice_df['Zipcode'].isin(zipcodes), :]
slice_zip = slice_zip.loc[slice_zip['Year-Month'] != '2019-04']
slice_zip['RentalPrice'] = slice_zip['Tuple'].apply(lambda x: rental_df.loc[int(x[0]),x[1]])
merge_df = pd.merge(slice_zip, crime_df, how='outer')
merge_df.to_csv('../LA_Data/LA_Crimes_Zipcodes_Rental_Prices', index=False)

/Users/mchifala/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
crime_df = pd.read_csv("../LA_Data/LA_Crimes_Zipcodes_Rental_Prices.csv")
crime_df['Crime_Type'] = crime_df['Crime Code Description'].apply(lambda x: dct[x])

/Users/mchifala/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


235

In [7]:
dct = {'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)': 'CRIMINAL DAMAGE',
'VEHICLE - STOLEN': 'MOTOR VEHICLE THEFT', 
'BURGLARY FROM VEHICLE': 'BURGLARY',
'VANDALISM - MISDEAMEANOR ($399 OR UNDER)':'CRIMINAL DAMAGE',
'INTIMATE PARTNER - SIMPLE ASSAULT': 'DOMESTIC VIOLENCE', 
'ROBBERY':'ROBBERY',
'DISCHARGE FIREARMS/SHOTS FIRED':'WEAPONS VIOLATION',
'CRIMINAL THREATS - NO WEAPON DISPLAYED':'INTIMIDATION',
'BATTERY - SIMPLE ASSAULT':'BATTERY', 
'BURGLARY':'BURGLARY', 
'BURGLARY, ATTEMPTED':'BURGLARY',
'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT':'ASSAULT',
'INTIMATE PARTNER - AGGRAVATED ASSAULT':'DOMESTIC VIOLENCE', 
'CONTEMPT OF COURT':'OTHER OFFENSE',
'SHOTS FIRED AT INHABITED DWELLING':'WEAPONS VIOLATION',
'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)':'THEFT',
'BRANDISH WEAPON':'INTIMIDATION', 
'ATTEMPTED ROBBERY':'ROBBERY',
'THEFT PLAIN - PETTY ($950 & UNDER)':'THEFT', 
'TRESPASSING':'CRIMINAL TRESPASS',
'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD':'OBSCENITY',
'OTHER MISCELLANEOUS CRIME':'OTHER OFFENSE', 
'THEFT, PERSON':'THEFT',
'VEHICLE - ATTEMPT STOLEN':'MOTOR VEHICLE THEFT',
'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH':'CRIM SEXUAL ASSAULT',
'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ':'CRIM SEXUAL ASSAULT',
'OTHER ASSAULT':'ASSAULT',
'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD':'THEFT',
'INDECENT EXPOSURE':'PUBLIC INDECENCY', 
'THEFT OF IDENTITY':'DECEPTIVE PRACTICE',
'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS':'HUMAN TRAFFICKING',
'THEFT FROM MOTOR VEHICLE - ATTEMPT':'THEFT', 
'BUNCO, PETTY THEFT':'THEFT',
'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)':'SEX OFFENSE',
'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)':'THEFT',
'BATTERY POLICE (SIMPLE)':'BATTERY',
'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)':'OFFENSE INVOLVING CHILDREN',
'DISTURBING THE PEACE':'PUBLIC PEACE VIOLATION', 
'DOCUMENT FORGERY / STOLEN FELONY':'DECEPTIVE PRACTICE',
'BATTERY WITH SEXUAL CONTACT':'CRIM SEXUAL ASSAULT', 
'CHILD ANNOYING (17YRS & UNDER)':'OFFENSE INVOLVING CHILDREN',
'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT':'OFFENSE INVOLVING CHILDREN', 
'CRIMINAL HOMICIDE':'HOMICIDE',
'RESISTING ARREST':'INTERFERENCE WITH PUBLIC OFFICER', 
'PURSE SNATCHING':'ROBBERY', 
'VIOLATION OF COURT ORDER':'OTHER OFFENSE',
'KIDNAPPING - GRAND ATTEMPT':'KIDNAPPING',
'SHOPLIFTING - PETTY THEFT ($950 & UNDER)':'THEFT',
'VIOLATION OF RESTRAINING ORDER':'OTHER OFFENSE',
'THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)':'THEFT',
'CHILD STEALING':'KIDNAPPING', 
'ARSON':'ARSON', 
'CRUELTY TO ANIMALS':'OTHER OFFENSE',
'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT':'OFFENSE INVOLVING CHILDREN', 
'BIKE - STOLEN':'THEFT',
'BURGLARY FROM VEHICLE, ATTEMPTED':'BURGLARY', 
'KIDNAPPING':'KIDNAPPING', 
'RAPE, FORCIBLE':'CRIM SEXUAL ASSAULT',
'VIOLATION OF TEMPORARY RESTRAINING ORDER':'OTHER OFFENSE', 
'FALSE IMPRISONMENT':'OTHER OFFENSE',
'CHILD NEGLECT (SEE 300 W.I.C.)':'OFFENSE INVOLVING CHILDREN', 
'ORAL COPULATION':'SEX OFFENSE',
'BUNCO, ATTEMPT':'GAMBLING', 
'LEWD CONDUCT':'PUBLIC INDECENCY', 
'STALKING':'STALKING', 
'PIMPING':'HUMAN TRAFFICKING',
'UNAUTHORIZED COMPUTER ACCESS':'OTHER OFFENSE', 
'BUNCO, GRAND THEFT':'THEFT',
'CREDIT CARDS, FRAUD USE ($950 & UNDER':'DECEPTIVE PRACTICE', 
'RECKLESS DRIVING':'OTHER OFFENSE',
'THEFT FROM PERSON - ATTEMPT':'THEFT', 
'BOMB SCARE':'INTIMIDATION',
'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $400 & UNDER':'DECEPTIVE PRACTICE',
'SEXUAL PENETRATION W/FOREIGN OBJECT':'CRIM SEXUAL ASSAULT',
'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE':'HUMAN TRAFFICKING',
'DRIVING WITHOUT OWNER CONSENT (DWOC)':'OTHER OFFENSE', 
'PEEPING TOM':'SEX OFFENSE',
'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT':'WEAPONS VIOLATION',
'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER':'ASSAULT',
'LEWD/LASCIVIOUS ACTS WITH CHILD':'OFFENSE INVOLVING CHILDREN', 
'LYNCHING - ATTEMPTED':'HOMICIDE',
'CREDIT CARDS, FRAUD USE ($950.01 & OVER)':'DECEPTIVE PRACTICE', 
'ILLEGAL DUMPING':'OTHER OFFENSE',
'THROWING OBJECT AT MOVING VEHICLE':'CRIMINAL DAMAGE', 
'FALSE POLICE REPORT':'DECEPTIVE PRACTICE',
'EXTORTION':'INTIMIDATION', 
'THEFT PLAIN - ATTEMPT':'THEFT', 
'FAILURE TO YIELD':'OTHER OFFENSE',
'DRUGS, TO A MINOR':'NARCOTICS', 
'SHOPLIFTING - ATTEMPT':'THEFT', 
'RAPE, ATTEMPTED':'CRIM SEXUAL ASSAULT',
'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)':'THEFT', 
'PANDERING':'OTHER OFFENSE',
'CONTRIBUTING':'OTHER OFFENSE', 
'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)':'THEFT',
'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $400':'DECEPTIVE PRACTICE',
'THEFT, COIN MACHINE - PETTY ($950 & UNDER)':'THEFT',
'THREATENING PHONE CALLS/LETTERS':'INTIMIDATION', 
'PICKPOCKET':'THEFT',
'CHILD PORNOGRAPHY':'SEX OFFENSE', 
'WEAPONS POSSESSION/BOMBING':'WEAPONS VIOLATION', 
'PROWLER':'OTHER OFFENSE',
'COUNTERFEIT':'DECEPTIVE PRACTICE', 
'DOCUMENT WORTHLESS ($200.01 & OVER)':'DECEPTIVE PRACTICE',
'CHILD ABANDONMENT':'OFFENSE INVOLVING CHILDREN', 
'THEFT, COIN MACHINE - ATTEMPT':'THEFT', 
'DRUNK ROLL':'LIQUOR LAW VIOLATION',
'BATTERY ON A FIREFIGHTER':'BATTERY', 
'PETTY THEFT - AUTO REPAIR':'THEFT',
'DISHONEST EMPLOYEE - PETTY THEFT':'THEFT', 
'BOAT - STOLEN':'MOTOR VEHICLE THEFT',
'DISHONEST EMPLOYEE - GRAND THEFT':'THEFT',
'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM':'CRIM SEXUAL ASSAULT',
'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)':'THEFT',
'TILL TAP - PETTY ($950 & UNDER)':'OTHER OFFENSE', 
'CONSPIRACY':'OTHER OFFENSE',
'GRAND THEFT / INSURANCE FRAUD':'THEFT', 
'TRAIN WRECKING':'OTHER OFFENSE',
'DISHONEST EMPLOYEE ATTEMPTED THEFT':'THEFT', 
'BIKE - ATTEMPTED STOLEN':'THEFT',
'PURSE SNATCHING - ATTEMPT':'THEFT', 
'INCITING A RIOT':'PUBLIC PEACE VIOLATION',
'BLOCKING DOOR INDUCTION CENTER':'OTHER OFFENSE', 
'BRIBERY':'DECEPTIVE PRACTICE',
'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)':'WEAPONS VIOLATION',
'DOCUMENT WORTHLESS ($200 & UNDER)':'DECEPTIVE PRACTICE', 
'DISRUPT SCHOOL':'PUBLIC PEACE VIOLATION',
'TILL TAP - GRAND THEFT ($950.01 & OVER)':'THEFT', 
'LYNCHING':'HOMICIDE',
'TELEPHONE PROPERTY - DAMAGE':'CRIMINAL DAMAGE', 
'PICKPOCKET, ATTEMPT':'THEFT',
'GRAND THEFT / AUTO REPAIR':'THEFT', 
'BIGAMY':'SEX OFFENSE', 
'ABORTION/ILLEGAL':'OTHER OFFENSE',
'TILL TAP - ATTEMPT':'OTHER OFFENSE', 
'FIREARMS RESTRAINING ORDER (FIREARMS RO)':'WEAPONS VIOLATION',
'MANSLAUGHTER, NEGLIGENT':'HOMICIDE', 
'FAILURE TO DISPERSE':'PUBLIC PEACE VIOLATION',
'FIREARMS TEMPORARY RESTRAINING ORDER (TEMP FIREARMS RO)':'WEAPONS VIOLATION' ,
'DRUNK ROLL - ATTEMPT':'LIQUOR LAW VIOLATION'
}